In [7]:
import mquad
from scipy.io import mmread, mmwrite
from scipy.sparse import csc_matrix
import numpy as np
import mquad
from mquad.mquad import Mquad
import pandas as pd
import os
from os import path
import matplotlib.pyplot as plt
from mquad.mquad_utils import findKnee
from scipy import sparse
import seaborn as sns


In [8]:
sample = 'D8'
input_folder = '/home/linxy29/data/maester/organoid/' + sample + '/manual_pipeline/filtered_SNP/'
output_folder = '/home/linxy29/data/maester/organoid/' + sample + '/manual_pipeline/further_mquad/'
cell_dat = {}
cell_dat['AD'] = csc_matrix(mmread(input_folder + 'passed_ad.mtx'))
cell_dat['DP'] = csc_matrix(mmread(input_folder + 'passed_dp.mtx'))
cell_dat['variants'] = np.genfromtxt(input_folder + "passed_variant_names.txt", dtype='str')

In [9]:
out_dir = output_folder
nproc = 32
minDP = 2
beta_mode = True
cutoff = None

In [10]:
mdphd = Mquad(AD = cell_dat['AD'], DP = cell_dat['DP'], variant_names = cell_dat['variants'])
df = mdphd.fit_deltaBIC(out_dir = out_dir, nproc = nproc, minDP = minDP, beta_mode = True)

2555 variants detected
variant names detected
CPUs used: 32
[MQuad] Initializing fit(mode: deltaBIC) on 2555 variants...
Cells qualified: 3397	model1 BIC:15941.13	model2 BIC:12530.70	 deltaBIC:3410.42Cells qualified: 3397	model1 BIC:15735.36	model2 BIC:11431.88	 deltaBIC:4303.48

Cells qualified: 3397	model1 BIC:16403.07	model2 BIC:12964.59	 deltaBIC:3438.47
Cells qualified: 3397	model1 BIC:16500.67	model2 BIC:12367.11	 deltaBIC:4133.56
Cells qualified: 3397	model1 BIC:17153.78	model2 BIC:15143.91	 deltaBIC:2009.87
Cells qualified: 3397	model1 BIC:16071.57	model2 BIC:12157.31	 deltaBIC:3914.26
Cells qualified: 3397	model1 BIC:15692.71	model2 BIC:11595.17	 deltaBIC:4097.54
Cells qualified: 3397	model1 BIC:15207.26	model2 BIC:11354.62	 deltaBIC:3852.65
Cells qualified: 3397	model1 BIC:16322.88	model2 BIC:13002.41	 deltaBIC:3320.47
Cells qualified: 3397	model1 BIC:17021.73	model2 BIC:16404.60	 deltaBIC:617.13
Cells qualified: 3397	model1 BIC:17484.72	model2 BIC:16646.15	 deltaBIC:838.57
C

In [11]:
def selectInformativeVariants(self, min_cells=2, export_heatmap=True, export_mtx=True, out_dir=None, existing_df=None, tenx_cutoff=None, numb_cutoff = None):
        #takes self.df, return best_ad and best_dp as array

        if existing_df is not None:
            #input /path/to/unsorted_debug_BIC_params.csv for existing df if model is already fit
            print('[MQuad] Fitted model detected, using' + existing_df + '...')
            self.df = pd.read_csv(existing_df)
            self.sorted_df = self.df.sort_values(by=['deltaBIC'], ascending=False)

        if out_dir is not None:
            if path.exists(out_dir) is not True:
                try:
                    os.mkdir(out_dir)
                except:
                    print("[MQuad] Can't make directory, do you have permission?")
        else:
            print('[MQuad] Out directory already exists, overwriting content inside...')
        
        if numb_cutoff > self.df.shape[0]:
            print('[MQuad] Number of variants to select exceeds number of variants in dataset, turning off numb mode...')
            numb_cutoff = None
        
        if tenx_cutoff is not None and numb_cutoff is not None:
            print('[MQuad] Both tenx and numb mode used, turning off numb mode...')
            numb_cutoff = None

        if tenx_cutoff is not None:
            print('[MQuad] Tenx mode used with cutoff = ' + str(tenx_cutoff))
            self.final_df = self.sorted_df[self.sorted_df.deltaBIC >= float(tenx_cutoff)]
            self.final_df = self.final_df[self.sorted_df.num_cells_minor_cpt >= min_cells]
        elif numb_cutoff is not None:
            print('[MQuad] Numb mode used with cutoff = ' + str(numb_cutoff))
            ## select numb_cutoff number of variants
            self.final_df = self.sorted_df.head(numb_cutoff)
            self.final_df = self.final_df[self.sorted_df.num_cells_minor_cpt >= min_cells]
        else:
            print('[MQuad] Finding knee point for deltaBIC cutoff...')
            #self.filt_df = self.sorted_df[self.sorted_df.deltaBIC >= 10]
            x,y,knee_x, cutoff = findKnee(self.df.deltaBIC)
            plt.plot(x, y)
            plt.axvline(x=knee_x, color="black", linestyle='--',label="cutoff")
            plt.legend()
            plt.ylabel("log10(\u0394BIC)")
            plt.xlabel("Cumulative probability")
            plt.savefig(out_dir + '/' + 'deltaBIC_cdf.pdf')

            print('deltaBIC cutoff = ', cutoff)
            #self.sorted_df['VALID'] = self.validateSNP(self.sorted_df.variant_name)
            self.sorted_df['PASS_KP'] = self.sorted_df.deltaBIC.apply(lambda x: True if x >= cutoff else False)
            self.sorted_df['PASS_MINCELLS'] = self.sorted_df.num_cells_minor_cpt.apply(lambda x: True if x >= min_cells else False)

            self.final_df = self.sorted_df[(self.sorted_df.PASS_KP == True) & (self.sorted_df.PASS_MINCELLS == True)]


        idx = self.final_df.index
        best_ad = self.ad[idx]
        best_dp = self.dp[idx]

        print('Number of variants passing threshold: '  + str(len(best_ad)))

        #fname = by + '_' + str(threshold) + '_'

        if self.variants is not None:
            best_vars = np.array(self.variants)[idx]
            #renamed_vars = []
            #for var in best_vars:
                #renamed_vars.append((var.split('_')[1] + var.split('_')[2] + '>' + var.split('_')[3]))

            with open(out_dir + '/' + 'passed_variant_names.txt', "w+") as var_file:
                #var_file.write('\n'.join(str(var) for var in renamed_vars))
                var_file.write('\n'.join(str(var) for var in best_vars))
                
        if export_heatmap:
            af = best_ad/best_dp
            #af = af.fillna(0)
            fig, ax = plt.subplots(figsize=(8,6))
            plt.title("Allele frequency of top variants")
            plt.style.use('seaborn-dark')
            pal = "YlGnBu"
            if self.variants is not None:
                sns.heatmap(af, cmap=pal, yticklabels=best_vars)
                #sns.heatmap(af, cmap=pal, yticklabels=renamed_vars)
                plt.yticks(rotation=0)
            else:
                sns.heatmap(af, cmap=pal)
                plt.yticks(rotation=0)
            plt.savefig(out_dir + '/' + 'top variants heatmap.pdf')

        #export ad dp mtx out for vireo
        if export_mtx is True:
            mmwrite(out_dir + '/' + 'passed_ad.mtx', sparse.csr_matrix(best_ad))
            mmwrite(out_dir + '/' + 'passed_dp.mtx', sparse.csr_matrix(best_dp))

        return best_ad, best_dp

In [12]:
best_ad, best_dp = selectInformativeVariants(self=mdphd, out_dir = out_dir, numb_cutoff = 2000, export_heatmap=False)

[MQuad] Numb mode used with cutoff = 2000


/tmp/ipykernel_25060/2251316436.py:35: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  self.final_df = self.final_df[self.sorted_df.num_cells_minor_cpt >= min_cells]


Number of variants passing threshold: 1999
